0. Importing Necessary libraries :

In [ ]:
import os
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt
from math import floor
import pandas as pd 
import cv2
import matplotlib.gridspec as gridspec
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers

from keras.initializers import RandomNormal
from keras.optimizers import Adam
from numpy.random import randn

import tensorflow_addons as tfa

from kaggle_datasets import KaggleDatasets



In [ ]:
BATCH_SIZE=2

In [ ]:
GCS_PATH=KaggleDatasets().get_gcs_path()

monet_filenames=tf.io.gfile.glob(str(GCS_PATH+'/monet_tfrec/*.tfrec'))
photo_filenames=tf.io.gfile.glob(str(GCS_PATH + '/photo_tfrec/*.tfrec'))

print('monet_filenames',len(monet_filenames))
print('photo_filenames',len(photo_filenames))


AUTOTUNE = tf.data.experimental.AUTOTUNE 
print(tf.__version__)



image_size=[256,256]
def decode_img(img):
    img=tf.image.decode_image(img,channels=3)
    img=(tf.cast(img,tf.float32)/127.5)-1
    img=tf.reshape(img,[*image_size,3])
    return img



def read_tfrecord(example):
    tfrecord_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_img(example['image'])
    return image

def load_dataset(filenames,labeled=True,ordered=False):
    dataset=tf.data.TFRecordDataset(filenames)
    dataset=dataset.map(read_tfrecord,num_parallel_calls=AUTOTUNE)
    return dataset


monet_ds=load_dataset(monet_filenames,labeled=True).batch(BATCH_SIZE)
photo_ds=load_dataset(photo_filenames,labeled=True).batch(BATCH_SIZE)










In [ ]:

def Plot(arr1,arr2,flag):

    arr1=next(iter(arr1))
    arr1 = (arr1 + 1) / 2.0
    fig, axes = plt.subplots(2, 2)
    fig.set_size_inches(10,6)
    count = 0
   
    for j in range(2):
            axes[0, j].imshow(arr1[count])
            axes[0, j].axis('off')
            count += 1

    if flag==True:    
        arr2=next(iter(arr2))
        arr2 = (arr2 + 1) / 2.0
    count = 0
    for i in range(2):
       
            axes[1, i].imshow(arr2[count])
            axes[1, i].axis('off')
            count += 1

    plt.tight_layout() 
    plt.show()
    


In [ ]:
Plot(monet_ds,photo_ds,True)

In [ ]:
Plot(monet_ds,photo_ds,True)

In [ ]:
def Generator():
  
    inputs = layers.Input(shape=[256,256,3,])
    
    init= RandomNormal(mean=0.0, stddev=0.02)
    gamma_init =keras.initializers.RandomNormal(mean=0.0, stddev=0.02)    
    
    conv1 = layers.Conv2D(32,4,strides=2,padding='same',kernel_initializer=init)(inputs)
    conv1=layers.BatchNormalization(gamma_initializer=gamma_init)(conv1)
    #conv1=layers.Dropout(0.5)(conv1)
    conv1=layers.LeakyReLU()(conv1)
    
    
    conv2 = layers.Conv2D(64,4,strides=2,padding='same',kernel_initializer=init)(conv1)
    conv2=layers.BatchNormalization(gamma_initializer=gamma_init)(conv2)
   # conv2=layers.Dropout(0.5)(conv2)
    conv2=layers.LeakyReLU()(conv2)
   
    
    conv3 = layers.Conv2D(128,4,strides=2,padding='same',kernel_initializer=init)(conv2)
    conv3=layers.BatchNormalization(gamma_initializer=gamma_init)(conv3)
   # conv3=layers.Dropout(0.5)(conv3)
    conv3=layers.LeakyReLU()(conv3)

    
    conv4 = layers.Conv2D(256,4,strides=2,padding='same',kernel_initializer=init)(conv3)
    conv4=layers.BatchNormalization(gamma_initializer=gamma_init)(conv4)
   # conv4=layers.Dropout(0.5)(conv4)
    conv4=layers.LeakyReLU()(conv4)
  
    
    
    conv5 = layers.Conv2D(512,4,strides=2,padding='same',kernel_initializer=init)(conv4)
    conv5=layers.BatchNormalization(gamma_initializer=gamma_init)(conv5)
    #conv5=layers.Dropout(0.5)(conv5)
    conv5=layers.LeakyReLU()(conv5)
    
    
     
    up1 = layers.Conv2DTranspose(256,4,strides=(2,2),padding='same',kernel_initializer=init)(conv5)
    up1=layers.BatchNormalization(gamma_initializer=gamma_init)(up1)
    #up1=layers.Dropout(0.5)(up1)
    up1=layers.LeakyReLU()(up1)
    merge1 = layers.concatenate([up1, conv4], axis=3) 
    

    
    up2 = layers.Conv2DTranspose(128,4,strides=(2,2),padding='same',kernel_initializer=init)(merge1)
    up2=layers.BatchNormalization(gamma_initializer=gamma_init)(up2)
    #up2=layers.Dropout(0.5)(up2)
    up2=layers.LeakyReLU()(up2)
    merge2 = layers.concatenate([up2, conv3], axis=3) 
    


    up3 = layers.Conv2DTranspose(64,4,strides=(2,2),padding='same',kernel_initializer=init)(merge2)
    up3=layers.BatchNormalization(gamma_initializer=gamma_init)(up3)
    up3=layers.Dropout(0.5)(up3)
    up3=layers.LeakyReLU()(up3)
    merge3 = layers.concatenate([up3, conv2], axis=3) 
   

   
    
    up4 = layers.Conv2DTranspose(32,4,strides=(2,2),padding='same',kernel_initializer=init)(merge3)
    up4=layers.BatchNormalization(gamma_initializer=gamma_init)(up4)
    up4=layers.Dropout(0.5)(up4)
    up4=layers.LeakyReLU()(up4)
    merge4 = layers.concatenate([up4, conv1], axis=3) 
    
   
 
    x = layers.Conv2DTranspose(3, 4, strides=(2,2),activation = 'tanh', padding = 'same')(merge4)  # tanh to get values between 1 and -1 same as monet images

    generator = keras.Model(inputs=inputs, outputs=x)

    generator.summary()

    return generator

In [ ]:
def Discrimnator():
    discriminator_input = layers.Input(shape=[256, 256, 3], name='input_image')
    init= RandomNormal(mean=0.0, stddev=0.02)
    
    x = layers.Conv2D(64, 4,kernel_initializer=init)(discriminator_input)
    #x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(128, 4, strides = 2,kernel_initializer=init)(x)
    #x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
  
    x = layers.Conv2D(128, 4, strides = 2,kernel_initializer=init)(x)
    #x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
  
    x = layers.Flatten()(x)
  
    x = layers.Dense(1, activation = 'sigmoid')(x)
  
    discriminator = keras.models.Model(discriminator_input, x)
    discriminator.summary()

    return discriminator


In [ ]:
generator=Generator()
discriminator=Discrimnator()

In [ ]:
intial=generator(next(iter(photo_ds)),training=False)
Plot(photo_ds,intial,False)

In [ ]:
class GAN(keras.Model):
    
    def __init__(self, gen, disc):
        super().__init__()
        self.gen = gen
        self.disc= disc

    def compile(self,gen_optimizer,disc_optimizer,gen_loss_fn,disc_loss_fn):
        super().compile()
        self.gen_optimizer = gen_optimizer
        self.disc_optimizer = disc_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn
       
        
    def train_step(self, batch_data):
        real_monet, real_photo = batch_data
        
        with tf.GradientTape(persistent=True) as tape:
       
            generated_images = self.gen(real_photo, training=True)
            
            real_output = self.disc(real_monet, training=True)
            fake_output = self.disc(generated_images, training=True)
            
            gen_loss = self.gen_loss_fn(fake_output)
            disc_loss =self.disc_loss_fn(real_output,fake_output)
            
       
        generator_gradients = tape.gradient(gen_loss,self.gen.trainable_variables)
        discriminator_gradients = tape.gradient(disc_loss,self.disc.trainable_variables)
        
        self.gen_optimizer.apply_gradients(zip(generator_gradients, self.gen.trainable_variables))
        self.disc_optimizer.apply_gradients(zip(discriminator_gradients,self.disc.trainable_variables))

        
        return {
            "gen_loss": gen_loss,
            "disc_loss":disc_loss
        }






In [ ]:
generator_optimizer = Adam(lr=0.0002, beta_1=0.5)
#generator.compile(optimizer = generator_optimizer, loss = 'categorical_crossentropy')

discriminator_optimizer = Adam(lr=0.0002, beta_1=0.5)
#discriminator.compile(optimizer = discriminator_optimizer,loss='binary_crossentropy')

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def Get_Noise_Batch():
    random_latent_vectors = randn(latent_dim * batch_size)
    # update to have the range [-1, 1]
    random_latent_vectors = -1 + random_latent_vectors * 2
    random_latent_vectors = random_latent_vectors.reshape((batch_size, latent_dim))
    
    return random_latent_vectors

In [ ]:
def Get_True_Batch(idx):
    real_images=original[idx:idx+batch_size]
    return real_images

In [ ]:
def G_loss(fake_output):
     return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
def D_loss(real_output,fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss+fake_loss

In [ ]:
model = GAN(generator,discriminator)
model.compile(generator_optimizer,discriminator_optimizer,G_loss,D_loss)


In [ ]:
model.fit(tf.data.Dataset.zip((monet_ds, photo_ds)),epochs=60)




In [ ]:
perd=generator(next(iter(photo_ds)),training=False)
Plot(photo_ds,perd,False)  

In [ ]:
import PIL
! mkdir ../images
i = 1

for img in photo_ds:
    prediction = generator(img, training=False).numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    
    im = PIL.Image.fromarray(prediction[0])
    im.save("../images/" + str(i) + ".jpg")
    i += 1
    im = PIL.Image.fromarray(prediction[1])
    im.save("../images/" + str(i) + ".jpg")
    i += 1
    
import shutil
shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")